<a href="https://colab.research.google.com/github/ggratani/embryo_segmentation/blob/main/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!git clone https://github.com/ggratani/embryo_segmentation.git

Cloning into 'embryo_segmentation'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 11 (delta 1), reused 5 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 5.05 KiB | 5.05 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
!pip install flask_cors

In [4]:
!pip install flask_ngrok

In [16]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import os
from flask import send_from_directory
import cv2
from flask_ngrok import run_with_ngrok

In [17]:
def format_label(clase, x, y, width, height, image_width, image_height):
    x_center = x / image_width
    y_center = y / image_height
    norm_width = width / image_width
    norm_height = height / image_height
    return f"{clase} {x_center} {y_center} {norm_width} {norm_height}\n".format(clase, x_center, y_center, norm_width, norm_height)


In [18]:
config = {
    # "DEBUG": True,
    "CACHE_TYPE": "SimpleCache",
    "CACHE_DEFAULT_TIMEOUT": 120
}

app = Flask(__name__)
CORS(run_with_ngrok(app))

# Configura la carpeta "images" como una carpeta estática
app.static_folder = r'C:\Users\gasto\Documents\React\embryoxite\backend_flask'

app.config.from_mapping(config)

@app.route('/alive')
def alive():
    print("alive")
    return jsonify({"mensaje": "Alive"})

In [19]:
# Configuracion ruta deteccion

UPLOAD_FOLDER_DETECTION = r'C:\Users\gasto\Documents\React\embryoxite\backend_flask\detection'

# Carpeta para las imágenes
app.config['IMAGE_UPLOAD_FOLDER_DETECTION'] = os.path.join(UPLOAD_FOLDER_DETECTION, 'images')
# Carpeta para las anotaciones
app.config['LABEL_UPLOAD_FOLDER_DETECTION'] = os.path.join(UPLOAD_FOLDER_DETECTION, 'labels')

# Configuracion ruta segmentation

UPLOAD_FOLDER_SEGMENTATION = r'embryo_segmentation/segmentation'

# Carpeta para las imágenes
app.config['IMAGE_UPLOAD_FOLDER_SEGMENTATION'] = os.path.join(UPLOAD_FOLDER_SEGMENTATION, 'images')
# Carpeta para las anotaciones
app.config['LABEL_UPLOAD_FOLDER_SEGMENTATION'] = os.path.join(UPLOAD_FOLDER_SEGMENTATION, 'labels')


In [20]:
@app.route('/save_image_detection', methods=['POST'])
def save_image():

    if 'image' not in request.files:
        return jsonify({"mensaje": "No se encontró el archivo en la solicitud"}), 400

    image = request.files['image']
    labels = request.files['labels'].read().decode('utf-8')

    print(labels)

    if image.filename == '':
        return jsonify({"mensaje": "No se seleccionó ningún archivo"}), 400

    if image and labels:
        # Verificar que la extensión del archivo sea válida (puedes personalizar esto según tus necesidades)
        allowed_extensions = {'png', 'jpg', 'jpeg', 'gif'}
        if '.' in image.filename and image.filename.rsplit('.', 1)[1].lower() in allowed_extensions:
            # Guardar el archivo en la carpeta de carga
            filename = os.path.join(app.config['IMAGE_UPLOAD_FOLDER_DETECTION'], image.filename)
            image.save(filename)

            # Supongamos que 'annotations' contiene las anotaciones en formato YOLO
            with open(os.path.join(app.config['LABEL_UPLOAD_FOLDER_DETECTION'], image.filename.rsplit('.', 1)[0] + ".txt"), 'w') as file:
                info = labels.split()
                x = (float(info[1]) + float(info[3]))/2
                y = (float(info[2]) + float(info[4]))/2

                width = float(info[3])-float(info[1])
                height = float(info[4])-float(info[2])
                file.write(format_label(info[0], x, y, float(width), float(height), float(info[5]), float(info[6])))

            return jsonify({"mensaje": "Imagen guardada con éxito"}), 200
        else:
            return jsonify({"mensaje": "Extensión de archivo no permitida"}), 400


In [21]:
@app.route('/save_image_segmentation', methods=['POST'])
def save_image_segmentation():

    if 'image' not in request.files:
        return jsonify({"mensaje": "No se encontró el archivo en la solicitud"}), 400

    image = request.files['image']
    image2 = request.files['image2']

    print(image)

    if image.filename == '':
        return jsonify({"mensaje": "No se seleccionó ningún archivo"}), 400

    if image:
        # Verificar que la extensión del archivo sea válida (puedes personalizar esto según tus necesidades)
        allowed_extensions = {'png', 'jpg', 'jpeg', 'gif'}
        if '.' in image.filename and image.filename.rsplit('.', 1)[1].lower() in allowed_extensions:
            # Guardar el archivo en la carpeta de carga
            filename = os.path.join(app.config['IMAGE_UPLOAD_FOLDER_SEGMENTATION'], image.filename)
            image.save(filename)

            # Decodifica la matriz de bytes en una imagen de OpenCV en escala de grises
            imagen_opencv = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

            # Define el umbral deseado (ajusta este valor según tus necesidades)
            umbral = 200

            # Aplica el umbral con cv2.THRESH_BINARY_INV
            _, imagen_umbralizada = cv2.threshold(imagen_opencv, umbral, 255, cv2.THRESH_BINARY)

            # Guarda la imagen umbralizada
            cv2.imwrite(os.path.join(app.config['LABEL_UPLOAD_FOLDER_SEGMENTATION'], image.filename), imagen_umbralizada)

            filename = os.path.join(app.config['IMAGE_UPLOAD_FOLDER_SEGMENTATION'], image.filename)
            image2.save(filename)

            return jsonify({"mensaje": "Imagen guardada con éxito"}), 200
        else:
            return jsonify({"mensaje": "Extensión de archivo no permitida"}), 400

In [ ]:
@app.route('/get_images_detection', methods=['GET'])
def get_images_detection():
    # Obtener la lista de nombres de archivo en el directorio de imágenes
    image_files = os.listdir(app.config['IMAGE_UPLOAD_FOLDER_DETECTION'])

    # Filtrar la lista para incluir solo archivos de imagen (por extensión)
    allowed_extensions = ['.jpg', '.jpeg', '.png', '.gif']
    image_files = [file for file in image_files if any(file.endswith(ext) for ext in allowed_extensions)]

    # Devolver la lista de nombres de archivo como respuesta JSON
    return jsonify({'images': image_files})

In [ ]:
@app.route('/get_images_segmentation', methods=['GET'])
def get_images_segmentation():
    # Obtener la lista de nombres de archivo en el directorio de imágenes
    image_files = os.listdir(app.config['IMAGE_UPLOAD_FOLDER_SEGMENTATION'])

    # Filtrar la lista para incluir solo archivos de imagen (por extensión)
    allowed_extensions = ['.jpg', '.jpeg', '.png', '.gif']
    image_files = [file for file in image_files if any(file.endswith(ext) for ext in allowed_extensions)]

    # Devolver la lista de nombres de archivo como respuesta JSON
    return jsonify({'images': image_files})

# Ruta para servir las imágenes estáticas
@app.route('/images/<path:filename>')
def serve_image(filename):
    print(filename)
    return send_from_directory(app.static_folder, filename)

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e5ba-34-133-40-112.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Oct/2023 02:38:35] "POST /save_image_segmentation HTTP/1.1" 200 -


<FileStorage: 'imagen_1697855913768.png' ('image/png')>


...

<FileStorage: 'imagen_1697857168079.png' ('image/png')>


INFO:werkzeug:127.0.0.1 - - [21/Oct/2023 02:59:45] "POST /save_image_segmentation HTTP/1.1" 200 -


<FileStorage: 'imagen_1697857183766.png' ('image/png')>


INFO:werkzeug:127.0.0.1 - - [21/Oct/2023 03:00:00] "POST /save_image_segmentation HTTP/1.1" 200 -


<FileStorage: 'imagen_1697857199293.png' ('image/png')>


In [14]:
!git remote add origin https://github.com/ggratani/embryo_segmentation.git

fatal: not a git repository (or any of the parent directories): .git
